## Initialization

In [1]:
# import findspark
import findspark
findspark.init()

In [2]:
# import SparkSession
from pyspark.sql import SparkSession

In [3]:
# create session
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

In [4]:
print(spark)

In [5]:
# read dataset
df = spark.read.csv("new_york_tree_census_1995.csv", header=True, inferSchema=True)

In [6]:
# find out dataset's length
df.count()

516989

In [7]:
# find out attributes
df.head()

Row(recordid=433600, address='73-031 57 AV', house_number='73-031', street='57 AV', zip_original=11378, cb_original=405, site='Front', species='QUPA', diameter=6, status='Good', wires='Yes', sidewalk_condition='Good', support_structure='None', borough='Queens', x=1015198.8, y=204725.3752, longitude=-73.888337, latitude=40.728546, cb_new=405, zip_new=11378, censustract_2010='49302', censusblock_2010='2000', nta_2010='QN30', segmentid=74525, spc_common='OAK PIN', spc_latin='QUERCUS PALUSTRIS', location='(40.728546 -73.888337)')

In [8]:
# find out each schema
df.schema

StructType(List(StructField(recordid,IntegerType,true),StructField(address,StringType,true),StructField(house_number,StringType,true),StructField(street,StringType,true),StructField(zip_original,IntegerType,true),StructField(cb_original,IntegerType,true),StructField(site,StringType,true),StructField(species,StringType,true),StructField(diameter,IntegerType,true),StructField(status,StringType,true),StructField(wires,StringType,true),StructField(sidewalk_condition,StringType,true),StructField(support_structure,StringType,true),StructField(borough,StringType,true),StructField(x,DoubleType,true),StructField(y,DoubleType,true),StructField(longitude,DoubleType,true),StructField(latitude,DoubleType,true),StructField(cb_new,IntegerType,true),StructField(zip_new,IntegerType,true),StructField(censustract_2010,StringType,true),StructField(censusblock_2010,StringType,true),StructField(nta_2010,StringType,true),StructField(segmentid,IntegerType,true),StructField(spc_common,StringType,true),StructFi

In [9]:
# create alias table to work on as 'trees'
df.createOrReplaceTempView("trees")

## Data Mining

### Query 1

In [10]:
# find the average diameter for each species
query1 = spark.sql("SELECT species, CONCAT(ROUND(AVG(diameter)), ' cm') AS average_diameter\
                    FROM trees\
                    GROUP BY species\
                    ORDER BY AVG(diameter) DESC")
query1.show()

+-------+----------------+
|species|average_diameter|
+-------+----------------+
|   FRQU|         39.0 cm|
|   QUFA|         33.0 cm|
|   CAOV|         31.0 cm|
|   SAMA|         31.0 cm|
|  POTR2|         29.0 cm|
|   QUMU|         25.0 cm|
|   SYCU|         25.0 cm|
|   ULPR|         24.0 cm|
|   ULPU|         24.0 cm|
|   CAGL|         22.0 cm|
|   PRAV|         22.0 cm|
|   QUAU|         21.0 cm|
|   AEOC|         21.0 cm|
|   LITU|         20.0 cm|
|   SAPE|         19.0 cm|
|   GRRO|         19.0 cm|
|   PLAC|         19.0 cm|
|   CAPU|         18.0 cm|
|   AEHI|         18.0 cm|
|   PLRU|         18.0 cm|
+-------+----------------+
only showing top 20 rows



### Query 2

In [11]:
# find status and total of status for each species and sort it by the total
query2 = spark.sql("SELECT species, status, COUNT(status) AS total\
                    FROM trees\
                    GROUP BY species, status\
                    ORDER BY species ASC, COUNT(status) DESC")
query2.show()

+-------+---------+-----+
|species|   status|total|
+-------+---------+-----+
|     AB|     Good|  138|
|     AB|Excellent|   34|
|     AB|     Poor|   16|
|     AB|     Dead|    4|
|     AB|  Unknown|    4|
|     AB|    Shaft|    2|
|   ABBA|     Good|    3|
|   ABBA|Excellent|    2|
|   ABCO|     Good|    3|
|     AC|     Good|  388|
|     AC|Excellent|  251|
|     AC|     Poor|  107|
|     AC|     Dead|   32|
|     AC|  Unknown|   17|
|     AC|     Fair|   11|
|     AC|    Shaft|    1|
|     AC|    Stump|    1|
|   ACAU|     Good|    4|
|   ACAU|Excellent|    1|
|   ACBU|     Good|    2|
+-------+---------+-----+
only showing top 20 rows



### Query 3

In [12]:
# find street with top 10 largest amount of good and excellent trees
query3 = spark.sql("SELECT street, COUNT(species) AS total\
                    FROM trees\
                    WHERE status IN ('Good', 'Excellent')\
                    GROUP BY street\
                    ORDER BY COUNT(species) DESC\
                    LIMIT 10")
query3.show()

+------------+-----+
|      street|total|
+------------+-----+
|    BROADWAY| 2034|
|       78 ST| 1215|
|       79 ST| 1186|
|        5 AV| 1167|
|     PARK AV| 1145|
|  BEDFORD AV| 1087|
|F LEWIS BLVD| 1062|
|       80 ST| 1054|
|       81 ST| 1017|
| LINDEN BLVD| 1004|
+------------+-----+



### Query 4

In [13]:
# find street with top 10 largest amount of poor, dead and unknown trees
query4 = spark.sql("SELECT street, COUNT(species) AS total\
                    FROM trees\
                    WHERE status IN ('Poor', 'Dead', 'Unknown')\
                    GROUP BY street\
                    ORDER BY COUNT(species) DESC\
                    LIMIT 10")
query4.show()

+---------------+-----+
|         street|total|
+---------------+-----+
|       BROADWAY|  394|
|    QUEENS BLVD|  333|
|GRAND CONCOURSE|  320|
|           2 AV|  199|
|           3 AV|  193|
|    LINDEN BLVD|  176|
|         E 7 ST|  173|
|           1 AV|  169|
|   EASTERN PKWY|  169|
|           5 AV|  147|
+---------------+-----+



### Query 5

In [14]:
# find species with required support structure and its total
query5 = spark.sql("SELECT species, support_structure, COUNT(species) AS total\
                    FROM trees\
                    WHERE support_structure <> 'None'\
                    GROUP BY species, support_structure\
                    ORDER BY species, COUNT(species) DESC")
query5.show()

+-------+-----------------+-----+
|species|support_structure|total|
+-------+-----------------+-----+
|     AC|   Stakes / Wires|   43|
|   ACCA|   Stakes / Wires|    1|
|   ACPA|   Stakes / Wires|   15|
|   ACPE|   Stakes / Wires|    5|
|   ACPL|   Stakes / Wires| 2572|
|   ACPL| Guard Strangling|    2|
|   ACPS|   Stakes / Wires|  193|
|   ACPS| Guard Strangling|    1|
|   ACRU|   Stakes / Wires|  979|
|   ACRU| Guard Strangling|    1|
|  ACSA1|   Stakes / Wires|  313|
|  ACSA1| Guard Strangling|    1|
|  ACSA2|   Stakes / Wires|  439|
|  ACSA2| Guard Strangling|    3|
|   AEHI|   Stakes / Wires|    6|
|   AIAL|   Stakes / Wires|  190|
|   AIAL| Guard Strangling|    2|
|   ALJU|   Stakes / Wires|    1|
|     BE|   Stakes / Wires|   20|
|   BENI|   Stakes / Wires|    1|
+-------+-----------------+-----+
only showing top 20 rows



### Query 6

In [15]:
# show species and where it's sited along with the total
query6 = spark.sql("SELECT species, site, COUNT(site) AS total\
                    FROM trees\
                    GROUP BY species, site\
                    ORDER BY species")
query6.show()

+-------+-------------+-----+
|species|         site|total|
+-------+-------------+-----+
|     AB|     Assigned|   11|
|     AB|         Side|   58|
|     AB|       Median|    2|
|     AB|     Adjacent|   26|
|     AB|        Front|   99|
|     AB|       Across|    2|
|   ABBA|       Across|    1|
|   ABBA|        Front|    2|
|   ABBA|         Side|    2|
|   ABCO|        Front|    1|
|   ABCO|     Adjacent|    1|
|   ABCO|         Side|    1|
|     AC|     Assigned|   23|
|     AC|       Across|   44|
|     AC|     Adjacent|  134|
|     AC|         Side|  191|
|     AC|Side/Adjacent|    1|
|     AC|       Median|    1|
|     AC|  Median/Side|    2|
|     AC|        Front|  407|
+-------+-------------+-----+
only showing top 20 rows



### Query 7

In [16]:
# show the address of tree and its species that touches wires (power line) along with its location
query7 = spark.sql("SELECT DISTINCT address, species, CONCAT(longitude, ', ', latitude) AS location\
                    FROM trees\
                    WHERE wires <> 'None'\
                    ORDER BY address")
query7.show()

+------------------+-------+--------------------+
|           address|species|            location|
+------------------+-------+--------------------+
|   0 ROCKAWAY FRWY|   PLAC|-73.760559, 40.60...|
|   0 ROCKAWAY FRWY|UNKDEAD|-73.760559, 40.60...|
|            1 2 AV|UNKNOWN|-73.991008, 40.67...|
|            1 2 AV|   QUPA|-73.991008, 40.67...|
|            1 2 AV|   QURU|-73.991008, 40.67...|
|            1 5 AV|UNKDEAD|-73.839771, 40.79...|
|       1 ADRIAN AV|   TICO|-73.912754, 40.87566|
|    1 ASPINWALL ST|   PLAC|-74.24948, 40.508272|
|     1 BELMONT TER|  ACSA1|-74.081925, 40.64403|
|        1 BROWN PL|   MORU|-74.067776, 40.60...|
|       1 BUTLER ST|   FRAM|-73.994541, 40.68...|
|     1 CAROLINA RD|   ACPL|            1.0, 1.0|
|     1 CARTERET ST|   PLAC|-74.24847, 40.508581|
|         1 CASE AV|   QUPA|-74.195214, 40.51...|
|         1 CASE AV|  ACSA1|-74.195214, 40.51...|
|1 COLUMBIA HEIGHTS|   GLTR|-73.999569, 40.69...|
|     1 COLUMBUS AV|   ACPS|-74.092817, 40.60...|


### Query 8

In [17]:
# show address and its location which has either good or excellent tree but not good sidewalk condition
query8 = spark.sql("SELECT DISTINCT address, species, sidewalk_condition, CONCAT(longitude, ', ', latitude) AS location\
                    FROM trees\
                    WHERE sidewalk_condition <> 'Good' AND status IN ('Good', 'Excellent')\
                    ORDER BY address")
query8.show()

+------------------+-------+------------------+--------------------+
|           address|species|sidewalk_condition|            location|
+------------------+-------+------------------+--------------------+
|        1 ASCAN AV|   SOJA|                NA|            1.0, 1.0|
|        1 ASCAN AV|   QURU|            Raised|            1.0, 1.0|
|        1 ASCAN AV|   QURU|                NA|            1.0, 1.0|
|        1 ASCAN AV|   PRMA|                NA|            1.0, 1.0|
|       1 BUTLER ST|   FRAM|            Raised|-73.994541, 40.68...|
|       1 BUTLER ST|   PLAC|            Raised|-73.994541, 40.68...|
|     1 CARTERET ST|   PLAC|            Raised|-74.24847, 40.508581|
| 1 CO OP CITY BLVD|   FRPE|                NA|            1.0, 1.0|
| 1 CO OP CITY BLVD|   GLTR|                NA|            1.0, 1.0|
|1 COLUMBIA HEIGHTS|   GLTR|            Raised|-73.999569, 40.69...|
|         1 E 21 ST|   GIBI|                NA|-73.990502, 40.74...|
|        1 E 233 ST|   QUPA|      

## Exporting the Results
#### Let's say we'd like to export the result of query8 above to CSV, either as partitioned ones or as a whole data.

In [18]:
# save the results of query8 to partitioned csv forms
query8.write \
      .option("header", "true") \
      .csv("file:///C:/Users/Nuzha Musyafira/Documents/github/big-data/result/query8.csv")

In [19]:
# import pandas
import pandas as pd

In [20]:
# convert the result into pandas dataframe then save it as single csv file
pd_query8 = query8.toPandas()
pd_query8.to_csv("C:/Users/Nuzha Musyafira/Documents/github/big-data/result/pd-query8.csv", index=False)